In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
n_users = 37
gpu_id = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
# Download the dataset
download_elec_dataset()

# Clean dataset
elec_data = clean_elec()

from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(elec_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(elec_data)

elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

Files are ready
Data loaded..
Data aggregated by hour: (26304, 370)


In [6]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# elec_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/electricity/electricity.csv', index_label='date')


In [7]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(elec_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.data_path

'electricity.csv'

In [8]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [9]:
test_data[0][0].shape

(96, 371)

In [10]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [11]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(10, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(10, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

# Setup WANDB

In [12]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Elec dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Elec dataset",
    # Hyperparams
    config={
        "dataset": "Elec370",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 370,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [13]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 34.68894410133362
Epoch: 2 cost time: 36.45457410812378
Epoch: 3 cost time: 34.13445329666138
Epoch: 4 cost time: 32.56888127326965
Epoch: 5 cost time: 34.17438197135925
Epoch: 1 cost time: 32.09558176994324
Epoch: 2 cost time: 32.05680513381958
Epoch: 3 cost time: 32.024245738983154
Epoch: 4 cost time: 31.901092052459717
Epoch: 5 cost time: 35.43361568450928
Epoch: 1 cost time: 45.04165554046631
Epoch: 2 cost time: 43.79860210418701
Epoch: 3 cost time: 44.56614017486572
Epoch: 4 cost time: 43.137893199920654
Epoch: 5 cost time: 48.70283102989197


Progress:   3%|▎         | 1/30 [10:30<5:04:50, 630.71s/it]

Epoch: 1 cost time: 45.934237241744995
Epoch: 2 cost time: 44.445760011672974
Epoch: 3 cost time: 43.672683238983154
Epoch: 4 cost time: 45.42060208320618
Epoch: 5 cost time: 44.88624405860901
Epoch: 1 cost time: 45.1038556098938
Epoch: 2 cost time: 46.94602346420288
Epoch: 3 cost time: 44.94026064872742
Epoch: 4 cost time: 44.26863741874695
Epoch: 5 cost time: 45.250282764434814
Epoch: 1 cost time: 44.34543776512146
Epoch: 2 cost time: 44.40358066558838
Epoch: 3 cost time: 45.23905301094055
Epoch: 4 cost time: 45.90857148170471
Epoch: 5 cost time: 85.078453540802


Progress:   7%|▋         | 2/30 [25:00<6:00:02, 771.53s/it]

...

Progress:  93%|█████████▎| 28/30 [29:15:08<1:53:35, 3407.64s/it]

Epoch: 1 cost time: 211.5407838821411
Epoch: 2 cost time: 210.9681956768036
Epoch: 3 cost time: 211.8253870010376
Epoch: 4 cost time: 211.16775822639465
Epoch: 5 cost time: 211.61272621154785
Epoch: 1 cost time: 211.46603727340698
Epoch: 2 cost time: 213.48965167999268
Epoch: 3 cost time: 212.50907349586487
Epoch: 4 cost time: 214.5305690765381
Epoch: 5 cost time: 211.1011679172516
Epoch: 1 cost time: 210.58139634132385
Epoch: 2 cost time: 212.21718454360962
Epoch: 3 cost time: 212.41357278823853
Epoch: 4 cost time: 211.59238362312317
Epoch: 5 cost time: 213.6138300895691


Progress:  97%|█████████▋| 29/30 [30:11:57<56:48, 3408.34s/it]  

Epoch: 1 cost time: 212.90618920326233
Epoch: 2 cost time: 209.63871002197266
Epoch: 3 cost time: 212.27269387245178
Epoch: 4 cost time: 212.86779236793518
Epoch: 5 cost time: 212.0769543647766
Epoch: 1 cost time: 212.1082866191864
Epoch: 2 cost time: 212.28877687454224
Epoch: 3 cost time: 212.36102080345154
Epoch: 4 cost time: 213.16665768623352
Epoch: 5 cost time: 213.15836668014526
Epoch: 1 cost time: 210.76384830474854
Epoch: 2 cost time: 210.7933211326599
Epoch: 3 cost time: 209.49010753631592
Epoch: 4 cost time: 210.91504430770874
Epoch: 5 cost time: 212.3227415084839


Progress: 100%|██████████| 30/30 [31:08:43<00:00, 3737.46s/it]
